# 学習（SSD）

準備（変換）した学習データを用いて、実際に SSD (Single Shot Multibox Detector) で学習を実行します。

## 準備

In [ ]:
import sys
sys.path.append(r'../models/research')
sys.path.append(r'../models/research/slim')

import glob
import json
import re
import tensorflow as tf
# from utils import dataset_util

import functools

In [ ]:
from object_detection import trainer
from object_detection.builders import dataset_builder
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.utils import dataset_util

↑警告が出ることがありますが気にしないでください。

## 設定読込

In [ ]:
configs = config_util.get_configs_from_pipeline_file('./ssd_mobilenet_v1_manabiya.pbtxt')

In [ ]:
model_config = configs['model']
train_config = configs['train_config']
input_config = configs['train_input_config']

### model 設定確認

In [ ]:
model_config

In [ ]:
model_config.ssd.num_classes

### 訓練 設定確認

In [ ]:
train_config

In [ ]:
train_config.num_steps

↑ デフォルトは2000ステップにしてありますが、所要時間目安は以下の通り：

+ `ml.p2.xlarge`（GPUインスタンス）で実行した場合、約48分で完了します。  
+ `ml.m4.xlarge`（CPUインスタンス）で実行した場合、約10時間（！）で完了します。  

ハンズオン時間内に終わるよう、適宜修正してください。

In [ ]:
# train_config.num_steps = 100

In [ ]:
train_config.num_steps

## 学習

### モデル

In [ ]:
model_fn = functools.partial(
    model_builder.build,
    model_config=model_config,
    is_training=True)

### 入力

In [ ]:
def get_next(config):
    return dataset_util.make_initializable_iterator(
        dataset_builder.build(config)).get_next()

create_input_dict_fn = functools.partial(get_next, input_config)

### その他学習パラメータ

In [ ]:
cluster_data = None
cluster = None
task_data = {'type': 'master', 'index': 0}
task_info = type('TaskSpec', (object,), task_data)

In [ ]:
# Parameters for a single worker.
ps_tasks = 0
worker_replicas = 1
worker_job_name = 'lonely_worker'
task = 0
is_chief = True
master = ''

In [ ]:
num_clones = 1
clone_on_cpu = False
train_dir = './train'

### 学習実行

In [ ]:
trainer.train(create_input_dict_fn, model_fn, train_config, master, task,
    num_clones, worker_replicas, clone_on_cpu, ps_tasks,
    worker_job_name, is_chief, train_dir)

↑最初に警告が出ますが気にしないでください。